## **LangChain**



- Framework para desarrollar apps basadas en LLMs

### Instalación

In [ ]:
! pip install langchain
! pip install --upgrade --quiet  langchain
! pip install langchain-community
! pip install langchainhub
! pip install langchain-openai
! pip install chromadb bs4

In [13]:
from dotenv import load_dotenv
load_dotenv()
import os
key = os.getenv("OPENAI_API_KEY")

### **Componentes : LLM**

- OpenAI: Chat Model

Inicia el modelo

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)


In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write a recipe using {ingredient}.")
chain = prompt | llm

In [14]:
response = chain.invoke({"ingredient": "potato"}).content

In [ ]:
print(response)

- OpenAI: Embeddings Model

In [90]:
from langchain_openai import OpenAIEmbeddings

In [91]:
# Use OpenAIEmbeddings and size 3
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [92]:
# Embed the text
text = response 
doc_result = embeddings.embed_documents([text])
doc_result[0][:5]


[-0.019610895550667983,
 -0.03274273617843824,
 -0.011946397332724516,
 0.040325479172545385,
 -0.01069963855112415]

In [93]:
# Embed the query
query = "How many ingredients will I need for this recipe?"
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.019610895550667983,
 -0.03274273617843824,
 -0.011946397332724516,
 0.040325479172545385,
 -0.01069963855112415]

### **Componentes : Chains**


### **Q&A : Indexing**

- Cargar texto (documentos, base de conocimiento)
- Trocear el texto para poder hacer llamadas a la API
- Almacenar el contenido

In [94]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader

In [95]:
# Load the documents

loader = PyPDFDirectoryLoader("documents/")
docs = loader.load()

In [96]:
print(docs[0].page_content)

   
Contact
diana.bisbe@gmail.com
www.linkedin.com/in/
dianadiazbisbe  (LinkedIn)
Top Skills
Semantic Kernel
.NET Framework
Back-End Development
Certifications
Power BI Data Modeling with DAX
Global AI Bootcamp Participant
Intro to GitHub Copilot Course
Intro to Generative AI Course
BI Dashboards with Power BI
CourseDiana Diaz Bisbe
Cloud Solutions Developer @ ENCAMINA | Computer Science
Madrid, Community of Madrid, Spain
Summary
Diana Diaz Bisbe is a software engineer dedicated to the world of
artificial intelligence. 
Her passion for technology and her ability to solve complex technical
problems are evident in her work, where she primarily uses Python
and C# to develop impactful applications. Diana holds a degree in
Computer Science from the University of Central Florida. She is an
avid advocate for lifelong learning and stands out as a speaker in the
artificial intelligence community. She has held roles as a developer in
image processing and recognition projects, as well as in the c

In [97]:
# Chunk and embed the documents

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())


In [98]:
# Retrieve and generate using the relevant snippets of the blog.
prompt_text = '''You are an assistant for question-answering tasks. Use the following pieces of
        retrieved context to answer the question. If you don't know the answer, just say 
        that you don't know. Use three sentences maximum and keep the answer concise.
        Question: {question} 

        Context: {context} 

        Answer:
        '''
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template(prompt_text)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [99]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
print (format_docs(docs))

In [101]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("Qué me puedes decir acerca del contexto?")

### **LangChain Components: Chains**

- secuencias de llamadas, ya sea a un LLM, una herramienta o un paso de preprocesamiento de datos
- LCEL facilita la creación de cadenas complejas a partir de componentes básicos

In [21]:
# prompt + model + output parser Chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Write a recipe using the ingredient {ingredient}")
model = ChatOpenAI(model_name= "gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

choco_recipe = chain.invoke({"ingredient": "chocolate"})

In [22]:
print (choco_recipe)

Triple Chocolate Brownies

Ingredients:
- 1 cup unsalted butter
- 2 cups granulated sugar
- 4 large eggs
- 1 teaspoon vanilla extract
- 1 cup all-purpose flour
- 1/2 cup cocoa powder
- 1/4 teaspoon salt
- 1 cup dark chocolate chips
- 1 cup milk chocolate chips
- 1 cup white chocolate chips

Instructions:
1. Preheat the oven to 350°F (175°C). Grease a 9x13 inch baking dish and set aside.
2. In a microwave-safe bowl, melt the butter. Let it cool slightly.
3. In a separate large mixing bowl, whisk together the sugar and melted butter.
4. Add the eggs one at a time, whisking well after each addition. Stir in the vanilla extract.
5. In a medium bowl, sift together the flour, cocoa powder, and salt. Gradually add this dry mixture to the wet ingredients, stirring until just combined.
6. Fold in the dark chocolate chips, milk chocolate chips, and white chocolate chips, reserving a small handful of each variety for topping.
7. Pour the batter into the prepared baking dish and spread it evenly. 

In [34]:
# The prompt component takes the user input, 
# which is then used to construct a PromptValue 
# after using the topic to construct the prompt.

prompt_value = prompt.invoke({"ingredient": "chocolate"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Write a recipe using the ingredient chocolate')])

In [35]:
prompt_value.messages[0].content

'Write a recipe using the ingredient chocolate'

In [36]:
# component takes the generated prompt, 
# and passes into the OpenAI LLM model for evaluation. 
# The generated output from the model is a ChatMessage object.

message = model.invoke(prompt_value)
message

AIMessage(content='Recipe: Decadent Chocolate Brownies\n\nIngredients:\n- 1 cup unsalted butter\n- 2 cups granulated sugar\n- 4 large eggs\n- 1 teaspoon vanilla extract\n- 1 cup all-purpose flour\n- 1/2 cup cocoa powder\n- 1/4 teaspoon salt\n- 1 cup chocolate chips (semi-sweet or dark)\n\nInstructions:\n1. Preheat your oven to 350°F (175°C). Grease a 9x13-inch baking dish and set aside.\n\n2. In a microwave-safe bowl, melt the butter. Allow it to cool slightly.\n\n3. In a large mixing bowl, combine the melted butter and granulated sugar. Mix well until fully incorporated.\n\n4. Add the eggs, one at a time, mixing well after each addition. Stir in the vanilla extract.\n\n5. In a separate bowl, sift together the all-purpose flour, cocoa powder, and salt. Gradually add the dry ingredients to the wet mixture, stirring until just combined. Do not overmix.\n\n6. Gently fold in the chocolate chips, reserving a handful for topping.\n\n7. Pour the batter into the prepared baking dish, spreading

In [37]:
message.content

'Recipe: Decadent Chocolate Brownies\n\nIngredients:\n- 1 cup unsalted butter\n- 2 cups granulated sugar\n- 4 large eggs\n- 1 teaspoon vanilla extract\n- 1 cup all-purpose flour\n- 1/2 cup cocoa powder\n- 1/4 teaspoon salt\n- 1 cup chocolate chips (semi-sweet or dark)\n\nInstructions:\n1. Preheat your oven to 350°F (175°C). Grease a 9x13-inch baking dish and set aside.\n\n2. In a microwave-safe bowl, melt the butter. Allow it to cool slightly.\n\n3. In a large mixing bowl, combine the melted butter and granulated sugar. Mix well until fully incorporated.\n\n4. Add the eggs, one at a time, mixing well after each addition. Stir in the vanilla extract.\n\n5. In a separate bowl, sift together the all-purpose flour, cocoa powder, and salt. Gradually add the dry ingredients to the wet mixture, stirring until just combined. Do not overmix.\n\n6. Gently fold in the chocolate chips, reserving a handful for topping.\n\n7. Pour the batter into the prepared baking dish, spreading it evenly. Sprink

In [38]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRecipe for Chocolate Fudge Brownies:\n\nIngredients:\n- 1 cup all-purpose flour\n- 1/2 cup unsalted butter\n- 1 cup granulated sugar\n- 1 teaspoon vanilla extract\n- 2 large eggs\n- 1/2 teaspoon baking powder\n- 1/2 teaspoon salt\n- 1/2 cup chocolate chips\n- 1/2 cup chopped walnuts (optional)\n- 1/2 cup unsweetened cocoa powder\n- 1/4 cup milk\n- 1/4 cup vegetable oil\n\nInstructions:\n\n1. Preheat your oven to 350°F (180°C) and line a 9x9 inch baking pan with parchment paper.\n\n2. In a medium-sized bowl, mix together the flour, baking powder, and salt. Set aside.\n\n3. In a large saucepan, melt the butter over medium heat. Once melted, remove from heat and stir in the sugar, vanilla extract, and cocoa powder.\n\n4. Add in the eggs, one at a time, mixing well after each addition.\n\n5. Gradually stir in the flour mixture until well combined.\n\n6. In a small saucepan, heat the milk and vegetable oil until just warm.\n\n7. Slowly'

In [ ]:
#  the output_parser component takes in a ChatMessage,
# and transforms this into a Python string,
# which is returned from the invoke method.

### **Tools**

- interfaces que un Agent puede usar para interactuar con el mundo.
    - Nombre
    - Descripción
    - Esquema JSON de cuáles son los datos de entrada
    - Función a llamar
    - Si el resultado debe ser devuelto directamente al usuario







In [ ]:
# The name, description, and JSON schema can be used the prompt the LLM so it knows 
# how to specify what action to take, 
# and then the function to call is equivalent to taking that action.

In [44]:
%pip install --upgrade --quiet  youtube_search

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Built in tools
from langchain.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

tool.run("john green, 5")


"['https://www.youtube.com/watch?v=IsEmm3i2BH0&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=R4peoHkXsJg&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=EQ3DakqVrvo&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=we_wLw-1fwg&pp=ygUKam9obiBncmVlbg%3D%3D', 'https://www.youtube.com/watch?v=NgDGlcxYrhQ&pp=ygUKam9obiBncmVlbg%3D%3D']"

In [3]:
tool.name

'youtube_search'

In [4]:
tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [ ]:
# Define Custom tools
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

# This @tool decorator is the simplest way to define a custom tool.
# The decorator uses the function name as the tool name by default

@tool
def multiply(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b



### **Agents**

- es la cadena responsable de decidir qué paso dar a continuación. Por lo general, esto funciona con un modelo de lenguaje, un mensaje y un output parser.
- el agent executor es el runtime de un agente. Esto es lo que realmente llama al agente, ejecuta las acciones que elige, devuelve los resultados de la acción al agente y repite.

In [14]:
# Build custom Agent

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [21]:
def is_palindrome(word):
    return word == word[::-1]

In [22]:
from langchain.agents import tool

# Define a custom tool
@tool
def is_palindrome_tool(word: str) -> bool:
    """Checks if a word is palindrome"""
    return is_palindrome(word)


is_palindrome_tool.invoke("abc")

False

In [24]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [25]:
tools = [is_palindrome_tool, get_word_length]

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [26]:
# How does the llm know what tools it can use
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

In [29]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [30]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
list(agent_executor.stream({"input": "+´{¿Cuántas letras hay en la palabra "chocolate"}"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5There are 5 letters in the word "eudca".

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_darbvI9EoDJEkHvV5hVScShY', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}]})], tool_call_id='call_darbvI9EoDJEkHvV5hVScShY')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_darbvI9EoDJEkHvV5hVScShY', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_darbvI9EoDJEkHvV5hVScShY', 'function': {'arguments': '{